In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
nameCSV = 'innis_new.csv'
df = pd.read_csv(str(nameCSV), sep=r'\s+', engine='c', header=0, na_filter=False, \
                               dtype=np.float, low_memory=False)

data = df.sort_values(by=['tt'])

lat = data.values[:,0]
lon = data.values[:,1]
wh = data.values[:,2]
tt = data.values[:,3]
qc = data.values[:,4]

In [3]:
netCDF4.num2date(tt[0:200],u'days since 1985-01-01 00:00:00 UTC')

array([datetime.datetime(1985, 3, 31, 8, 13, 38, 15623),
       datetime.datetime(1985, 3, 31, 8, 13, 38, 992192),
       datetime.datetime(1985, 3, 31, 8, 13, 39, 976566),
       datetime.datetime(1985, 3, 31, 8, 13, 40, 953124),
       datetime.datetime(1985, 3, 31, 8, 13, 41, 929683),
       datetime.datetime(1985, 3, 31, 8, 13, 42, 914067),
       datetime.datetime(1985, 3, 31, 8, 13, 43, 890626),
       datetime.datetime(1985, 3, 31, 8, 13, 44, 875000),
       datetime.datetime(1985, 3, 31, 8, 13, 45, 851559),
       datetime.datetime(1985, 3, 31, 8, 13, 46, 835933),
       datetime.datetime(1985, 3, 31, 8, 13, 47, 812502),
       datetime.datetime(1985, 3, 31, 8, 13, 48, 789061),
       datetime.datetime(1985, 3, 31, 8, 13, 49, 773435),
       datetime.datetime(1985, 3, 31, 8, 13, 50, 750003),
       datetime.datetime(1985, 3, 31, 8, 13, 51, 734378),
       datetime.datetime(1985, 3, 31, 8, 13, 52, 710936),
       datetime.datetime(1985, 3, 31, 8, 13, 53, 695310),
       datetime

In [23]:
#TEST TO SEE ALL HIGH VALUES
idd = np.where(np.logical_and(wh>15, wh<30))[0]

for k in range(len(idd)):
    print wh[idd[k]], netCDF4.num2date(tt[idd[k]],u'days since 1985-01-01 00:00:00 UTC')

24.345001220703125 2013-02-18 14:34:14.718752
15.939001083374025 2018-02-16 21:52:05.874998
23.207000732421875 2019-02-17 07:54:46.398436
22.24700164794922 2019-02-17 07:54:47.437496
22.306001663208008 2019-02-17 07:54:48.476567
22.29300117492676 2019-02-17 07:54:49.515628
23.23900032043457 2019-02-17 07:54:50.554689
21.66200065612793 2019-02-17 07:54:51.593750
21.136001586914066 2019-02-17 07:54:52.632811
21.374000549316406 2019-02-17 07:54:53.679687
19.315000534057614 2019-02-17 20:19:27.749996
19.77800178527832 2019-02-17 20:19:28.796872
21.251001358032227 2019-02-17 20:19:29.843749
19.913000106811523 2019-02-17 20:19:30.875004
16.872001647949222 2019-02-17 20:19:31.921870
20.554000854492188 2019-02-17 20:19:32.968747
20.696001052856445 2019-02-17 20:19:34.000002
21.117000579833984 2019-02-17 20:19:35.046879
21.38900184631348 2019-02-17 20:19:36.078124
21.13800048828125 2019-02-17 20:19:37.125000
21.517000198364254 2019-02-17 20:19:38.171877
18.450000762939453 2019-02-17 20:19:39.20

In [6]:
tt1 = netCDF4.num2date(tt,u'days since 1985-01-01 00:00:00 UTC')
print tt1[0], wh[0]
print netCDF4.num2date(max(tt),u'days since 1985-01-01 00:00:00 UTC')

1985-03-31 08:13:38.015623 1.90500009059906
2019-02-20 11:54:42.070312


In [7]:
#MAKE MEAN SATELLITE TRACK. DOESNT SEEM TO WORK??
days = [] #daystart
for k in range(len(tt)):
       
    t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
    if k == 0: #get day 0. Then else: every day after that
        days.append(0)
        dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days.append(k)
            it += 1
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')

In [8]:
print netCDF4.num2date(tt[days[0]],u'days since 1985-01-01 00:00:00 UTC')#same as first record

# meanwh is the mean wh of each day (satellite track)
meanwh = []

for k in range(len(days)-1):
    meanwh.append(np.mean(wh[days[k]:days[k+1]])) 
    
print np.mean(meanwh), 'mean from each satellite track'
print np.mean(wh), 'mean from each record'
print len(wh), 'number of records'
print len(meanwh), 'number of satellite tracks'

#slightly different mean wh depending on if done by day, or individual records (each second)

1985-03-31 08:13:38.015623
1.4183269924489301 mean from each satellite track
1.4323041752714756 mean from each record
49180 number of records
2540 number of satellite tracks


In [9]:
netCDF4.num2date(days,u'days since 1985-01-01 00:00:00 UTC')

array([real_datetime(1985, 1, 1, 0, 0), real_datetime(1985, 1, 19, 0, 0),
       real_datetime(1985, 1, 23, 0, 0), ...,
       real_datetime(2119, 6, 4, 0, 0), real_datetime(2119, 6, 29, 0, 0),
       real_datetime(2119, 7, 19, 0, 0)], dtype=object)

In [10]:
len(days)

2541

MEAN MONTHLY VALUES

In [22]:
for k in range (len(days)):
    print netCDF4.num2date(days[k],u'days since 1985-01-01 00:00:00 UTC')

1985-01-01 00:00:00
1985-01-19 00:00:00
1985-01-23 00:00:00
1985-01-24 00:00:00
1985-01-29 00:00:00
1985-02-03 00:00:00
1985-02-09 00:00:00
1985-02-18 00:00:00
1985-03-01 00:00:00
1985-03-11 00:00:00
1985-03-12 00:00:00
1985-03-16 00:00:00
1985-03-27 00:00:00
1985-04-14 00:00:00
1985-04-23 00:00:00
1985-05-03 00:00:00
1985-05-08 00:00:00
1985-05-22 00:00:00
1985-05-31 00:00:00
1985-06-16 00:00:00
1985-07-05 00:00:00
1985-07-12 00:00:00
1985-07-25 00:00:00
1985-07-29 00:00:00
1985-08-15 00:00:00
1985-08-31 00:00:00
1985-09-18 00:00:00
1985-09-30 00:00:00
1985-10-02 00:00:00
1985-10-04 00:00:00
1985-10-06 00:00:00
1985-10-12 00:00:00
1985-10-15 00:00:00
1985-10-22 00:00:00
1985-11-01 00:00:00
1985-11-03 00:00:00
1985-11-09 00:00:00
1985-11-11 00:00:00
1985-11-18 00:00:00
1985-11-24 00:00:00
1985-11-26 00:00:00
1985-12-03 00:00:00
1985-12-04 00:00:00
1985-12-06 00:00:00
1985-12-12 00:00:00
1985-12-19 00:00:00
1985-12-26 00:00:00
1985-12-31 00:00:00
1986-01-07 00:00:00
1986-01-14 00:00:00


2053-07-05 00:00:00
2053-07-25 00:00:00
2053-08-09 00:00:00
2053-09-10 00:00:00
2053-09-30 00:00:00
2053-10-20 00:00:00
2053-11-06 00:00:00
2053-11-20 00:00:00
2053-12-10 00:00:00
2053-12-25 00:00:00
2054-01-14 00:00:00
2054-02-03 00:00:00
2054-02-23 00:00:00
2054-03-10 00:00:00
2054-03-29 00:00:00
2054-04-16 00:00:00
2054-05-06 00:00:00
2054-05-26 00:00:00
2054-06-12 00:00:00
2054-07-02 00:00:00
2054-07-22 00:00:00
2054-08-06 00:00:00
2054-08-21 00:00:00
2054-09-10 00:00:00
2054-10-01 00:00:00
2054-10-18 00:00:00
2054-11-07 00:00:00
2054-12-12 00:00:00
2054-12-23 00:00:00
2055-01-03 00:00:00
2055-01-23 00:00:00
2055-02-12 00:00:00
2055-02-27 00:00:00
2055-03-19 00:00:00
2055-04-23 00:00:00
2055-05-10 00:00:00
2055-05-30 00:00:00
2055-06-16 00:00:00
2055-07-06 00:00:00
2055-07-21 00:00:00
2055-08-02 00:00:00
2055-08-22 00:00:00
2055-09-11 00:00:00
2055-10-01 00:00:00
2055-10-21 00:00:00
2055-11-07 00:00:00
2055-12-15 00:00:00
2056-01-04 00:00:00
2056-01-12 00:00:00
2056-02-01 00:00:00


In [12]:
netCDF4.num2date(meanwh,u'days since 1985-01-01 00:00:00 UTC')

array([real_datetime(1985, 1, 3, 3, 53, 7, 209306),
       real_datetime(1985, 1, 1, 6, 59, 2, 401385),
       real_datetime(1985, 1, 3, 3, 56, 9, 602966), ...,
       real_datetime(1985, 1, 2, 10, 59, 31, 204917),
       real_datetime(1985, 1, 21, 19, 56, 38, 484389),
       real_datetime(1985, 1, 2, 4, 5, 5, 285073)], dtype=object)

In [16]:
janWh = []
febWh = []
marWh = []
aprWh = []
mayWh = []
junWh = []
julWh = []
augWh = []
sepWh = []
octWh = []
novWh = []
decWh = []

for k in range(len(days)):
    dt = netCDF4.num2date(days[k],u'days since 1985-01-01 00:00:00 UTC')
    if dt.month == 1:
        janWh.append(k)
    if dt.month == 2:
        febWh.append(k)
    if dt.month == 3:
        marWh.append(k)
    if dt.month == 4:
        aprWh.append(k)
    if dt.month == 5:
        mayWh.append(k)
    if dt.month == 6:
        junWh.append(k)
    if dt.month == 7:
        julWh.append(k)
    if dt.month == 8:
        augWh.append(k)
    if dt.month == 9:
        sepWh.append(k)
    if dt.month == 10:
        octWh.append(k)
    if dt.month == 11:
        novWh.append(k)
    if dt.month == 12:
        decWh.append(k)

In [17]:
febWh

[5,
 6,
 7,
 52,
 53,
 54,
 55,
 74,
 75,
 95,
 114,
 115,
 135,
 156,
 175,
 176,
 194,
 195,
 214,
 215,
 235,
 253,
 270,
 271,
 272,
 290,
 291,
 312,
 332,
 350,
 351,
 368,
 369,
 386,
 387,
 406,
 407,
 423,
 441,
 442,
 460,
 461,
 481,
 482,
 499,
 500,
 516,
 533,
 534,
 552,
 572,
 588,
 607,
 626,
 643,
 644,
 665,
 666,
 686,
 706,
 725,
 765,
 785,
 806,
 825,
 826,
 845,
 846,
 865,
 885,
 905,
 921,
 939,
 957,
 976,
 977,
 978,
 1011,
 1012,
 1040,
 1041,
 1077,
 1078,
 1079,
 1114,
 1143,
 1144,
 1172,
 1173,
 1174,
 1204,
 1205,
 1206,
 1207,
 1234,
 1235,
 1254,
 1255,
 1270,
 1271,
 1289,
 1290,
 1310,
 1311,
 1348,
 1368,
 1388,
 1408,
 1426,
 1442,
 1443,
 1461,
 1462,
 1481,
 1482,
 1499,
 1500,
 1516,
 1517,
 1533,
 1534,
 1547,
 1563,
 1579,
 1596,
 1610,
 1611,
 1629,
 1630,
 1631,
 1648,
 1649,
 1666,
 1667,
 1684,
 1703,
 1720,
 1738,
 1739,
 1756,
 1757,
 1773,
 1774,
 1792,
 1808,
 1825,
 1841,
 1842,
 1859,
 1872,
 1873,
 1889,
 1890,
 1906,
 1907,
 1924

In [20]:
janW = wh[janWh]
febW = wh[febWh]
marW = wh[marWh]
aprW = wh[aprWh]
mayW = wh[mayWh]
junW = wh[junWh]
julW = wh[julWh]
augW = wh[augWh]
sepW = wh[sepWh]
octW = wh[octWh]
novW = wh[novWh]
decW = wh[decWh]


print 'Month    Min             Max             Mean'
print 'Jan', janW.min(),janW.max(),janW.mean()
print 'Feb', febW.min(),febW.max(),febW.mean()
print 'Mar', marW.min(),marW.max(),marW.mean()
print 'Apr', aprW.min(),aprW.max(),aprW.mean()
print 'May', mayW.min(),mayW.max(),mayW.mean()
print 'Jun', junW.min(),junW.max(),junW.mean()
print 'Jul', julW.min(),julW.max(),julW.mean()
print 'Aug', augW.min(),augW.max(),augW.mean()
print 'Sep', sepW.min(),sepW.max(),sepW.mean()
print 'Oct', octW.min(),octW.max(),octW.mean()
print 'Nov', novW.min(),novW.max(),novW.mean()
print 'Dec', decW.min(),decW.max(),decW.mean()

Month    Min             Max             Mean
Jan 0.3850000202655792 3.1000001430511475 1.5311443014786794
Feb 0.3580000102519989 3.4170000553131104 1.5204924626398812
Mar 0.3580000102519989 3.627000093460083 1.5879695022470566
Apr 0.3580000102519989 4.1550002098083505 1.6243700757622719
May 0.3270000219345093 3.733000278472901 1.5544395322489633
Jun 0.27900001406669617 3.627000093460083 1.5694360860151138
Jul 0.25 3.733000278472901 1.4918843280110095
Aug 0.25 4.060000419616698 1.545322503302699
Sep 0.3370000123977661 4.743000030517578 1.594051352983866
Oct 0.4620000123977661 4.828999996185303 1.5763088294437952
Nov 0.3750000298023224 5.050000190734863 1.5293254274785804
Dec 0.31700000166893005 3.733000278472901 1.5389550265159693


In [ ]:
#why does this give me different mean wh value?

wh_ = wh[days]
qc_ = qc[days]
lon_ = lon[days]
tt_ = tt[days]
lat_ = lat[days]

print max(wh_)
print min(wh_)
print len(tt)
print len(days)
print np.mean(wh_)

In [ ]:
#for k in range(len(days)):
#    if wh_[k]>5:
#        print wh_[k], netCDF4.num2date(tt_[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
#This should be average per track, wh above 7.

for k in range(len(wh_)):
    if wh_[k]>7:
        print wh_[k], netCDF4.num2date(tt_[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
daywh = [] #daystart
for k in range(len(tt)):
    if wh[k]<10:
        t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        if k == 0: #get day 0. Then else: every day after that
            daywh.append(0)
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
            it = 0
        else:
            if t1.day != dd.day: #if day 1 is not equal to day 2, then append
                #print dd.day,t1.day
                daywh.append(k)
                it += 1
                dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
len(daywh)

In [ ]:
wh2 = wh[daywh]
qc2 = qc[daywh]
lon2 = lon[daywh]
tt2 = tt[daywh]
lat2 = lat[daywh]

print max(wh2)
print min(wh2)
print len(tt2)
print np.mean(wh2)

In [ ]:
for k in range(len(wh2)):
    if wh2[k]>7:
        print wh2[k], netCDF4.num2date(tt2[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
netCDF4.num2date(12087.8427231626124,u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
print np.mean(wh2), 'mean'
print max(wh2), 'max'
print min(wh2), 'min'

# for p in range(len(timing)):
            #print(u,' - ',k,'/',len(urlON),' - ',p,'/',len(timing))
            if timing[p] >= start_date and timing[p] <= end_date:
                if lats[p]>latmin and lats[p]<latmax and lons[p]>lonmin and lons[p]<lonmax:
                    if np.where(np.logical_and(0<wh[p]<10,qc[p]==1)):
                        boxLat.append(lats[p])